data modeling using python and postgres

check out the readme for detailed information about this project!

In [3]:
# importing libraries
import psycopg2
import pandas as pd

In [47]:
# reading our csv file
employee_df = pd.read_csv('employee.csv')

In [48]:
employee_df.head()

,id,name,phone,email,address
0,1,Bart Thornton,4-088-046-2812,Bart_Thornton9250@urn0m.business,"Maple Rue, 3966"
1,2,Michael Harper,2-054-128-1477,Michael_Harper6871@urn0m.meet,"Cavell Drive, 8517"
2,3,Chuck Brooks,3-348-241-6453,Chuck_Brooks3347@lyvnc.info,"Arctic Hill, 5136"
3,4,Abdul Healy,4-460-351-8807,Abdul_Healy4473@dbxli.store,"Colombo Crossroad, 5458"
4,5,Ethan Reid,0-258-347-4544,Ethan_Reid4292@avn7d.info,"Paris Crossroad, 6777"


In [49]:
# reading our second csv file
company_df = pd.read_csv('company.csv')

In [50]:
company_df.head()

,employee_id,company_id,company_name,location
0,1,0efc48df-70d1-4c6a-8130-b0ca3f582570,UPC,"Carlton Hill, 3896"
1,2,ca42e4f3-849b-49f5-8653-a266fe6d02fe,UPC,"Timothy Grove, 6591"
2,3,6846a865-ddc6-45c1-8ea2-6992d50c7b97,Team Guard SRL,"Bempton Road, 1640"
3,4,00ce762f-556d-4551-94b5-1c7c4c5ac123,Vodafone,"Chester Tunnel, 9206"
4,5,e5d431d1-87a1-46db-a6b4-3ac06caa153c,Facebook,"Sheffield Alley, 8271"


In [51]:
# and our third!
company_business_df = pd.read_csv('company_business.csv')

In [52]:
company_business_df.head()

,company_id,business
0,0efc48df-70d1-4c6a-8130-b0ca3f582570,Materials
1,ca42e4f3-849b-49f5-8653-a266fe6d02fe,Energy
2,6846a865-ddc6-45c1-8ea2-6992d50c7b97,Technology
3,00ce762f-556d-4551-94b5-1c7c4c5ac123,Technology
4,e5d431d1-87a1-46db-a6b4-3ac06caa153c,Consumer Discretionary


In [25]:
# writing a function that will create a connection to our pg db

def connect_db():
    # the db was already created in pgadmin
    conn = psycopg2.connect("host=127.0.0.1 dbname=employee user=postgres password=postgres")
    conn.set_session(autocommit = True)
    cur = conn.cursor()
    
    return cur, conn

In [27]:
# calling our function here
cur, conn = connect_db()

In [28]:
# we need to create 3 tables for our 3 csv files
# the below script is the code for creating the first table

employee_table = ("""CREATE TABLE IF NOT EXISTS employee(
id INT PRIMARY KEY,
name VARCHAR,
phone VARCHAR,
email VARCHAR,
address VARCHAR
)""")

In [29]:
# executing our newly created table and committing
cur.execute(employee_table)
conn.commit()

In [42]:
# and creating the second table
# here we are creating an ER - Entity Relationship
# ER between company and employee table!

company_table = ("""CREATE TABLE IF NOT EXISTS company(
employee_id INT, FOREIGN KEY (employee_id) REFERENCES employee (id),
company_id VARCHAR PRIMARY KEY,
company_name VARCHAR,
location VARCHAR
)""")

In [43]:
# executing and committing
cur.execute(company_table)
conn.commit()

In [44]:
# and creating the third table
# an ER here between company business and company table

company_business = ("""CREATE TABLE IF NOT EXISTS company_business(
company_id VARCHAR, FOREIGN KEY (company_id) REFERENCES company (company_id),
business VARCHAR
)""")

In [45]:
# executing and committing
cur.execute(company_business)
conn.commit()

In [46]:
# after creating those tables, we are writing sql using wildcards
# we will iterate and push our df after this

employee_table_import = ("""INSERT INTO employee(
id,
name,
phone,
email,
address)
VALUES (%s, %s, %s, %s, %s)
""") 

In [53]:
# this is not the best practice to do in a production environment!
# as the data is small, it's not a problem here

# we are iterating through each row in our dataframe and executing
for i, row in employee_df.iterrows():
    cur.execute(employee_table_import, list(row))

In [54]:
# committing our changes
conn.commit()

In [57]:
# same here! writing sql for our second table

company_table_import = ("""INSERT INTO company(
employee_id,
company_id,
company_name,
location)
VALUES (%s, %s, %s, %s)
""")

In [58]:
# and iterating and pushing!
for i, row in company_df.iterrows():
    cur.execute(company_table_import, list(row))

In [59]:
conn.commit()

In [60]:
# and for the third table!

company_business_table_import = ("""INSERT INTO company_business(
company_id,
business)
VALUES (%s, %s)
""")

In [61]:
for i, row in company_business_df.iterrows():
    cur.execute(company_business_table_import, list(row))

In [62]:
# commiting our transactions!
conn.commit()